In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import pandas as pd

In [14]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA(GPU)를 사용합니다.")
else:
    device = torch.device("cpu")
    print("CUDA(GPU)를 사용할 수 없으므로, CPU를 사용합니다.")

CUDA(GPU)를 사용합니다.


In [15]:
data = pd.read_csv(r'./dataset/styles.csv', on_bad_lines = 'skip')

In [16]:
data

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt
...,...,...,...,...,...,...,...,...,...,...
44419,17036,Men,Footwear,Shoes,Casual Shoes,White,Summer,2013.0,Casual,Gas Men Caddy Casual Shoe
44420,6461,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2011.0,Casual,Lotto Men's Soccer Track Flip Flop
44421,18842,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011.0,Casual,Puma Men Graphic Stellar Blue Tshirt
44422,46694,Women,Personal Care,Fragrance,Perfume and Body Mist,Blue,Spring,2017.0,Casual,Rasasi Women Blue Lady Perfume


In [17]:
df2 = data[data['subCategory'].isin(['Topwear', 'Shoes', 'Bags','Bottomwear', 'Watches', 'Innerwear', 'Eyewear', 'Wallets', 'Belts'])]

In [18]:
label_df = df2[['id','subCategory']]
label_df

,id,subCategory
0,15970,Topwear
1,39386,Bottomwear
2,59263,Watches
3,21379,Bottomwear
4,53759,Topwear
...,...,...
44417,12544,Topwear
44418,42234,Topwear
44419,17036,Shoes
44421,18842,Topwear


In [50]:
import os
from torchvision.datasets.folder import default_loader
from torch.utils.data import Dataset

class SingleImageFolder(Dataset):
    def __init__(self, root, transform=None, target_transform=None, loader=default_loader):
        self.root = root
        self.loader = loader
        self.transform = transform
        self.target_transform = target_transform

        self.imgs = self._make_dataset(root)

    def _make_dataset(self, dir):
        images = []
        for root, _, fnames in sorted(os.walk(dir)):
            for fname in fnames:
                path = os.path.join(root, fname)
                item = (path, 0)  # 클래스 없이 이미지를 하나의 클래스로 취급
                images.append(item)
        return images

    def __getitem__(self, index):
        path, target = self.imgs[index]
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return img, target

    def __len__(self):
        return len(self.imgs)

# 데이터셋 폴더 경로
data_folder = "./dataset/images"


In [77]:
from torch.utils.data import random_split

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((64, 64)),
    transforms.Normalize([0.8542, 0.8367, 0.8306], [0.2618, 0.2745, 0.2787]) # CIFAR10의 평균과 표준편차
])

# 데이터셋 불러오기
img_dataset = SingleImageFolder(root=data_folder, transform=transform)

# # DataLoader를 사용하여 이미지 데이터를 배치 단위로 불러올 수 있음
# img_data_loader = torch.utils.data.DataLoader(img_dataset, batch_size=128, shuffle=False)

# 데이터셋을 훈련(train) 및 테스트(test) 세트로 나누는 비율 설정
train_size = int(0.8 * len(img_dataset))
test_size = len(img_dataset) - train_size

# 데이터셋을 무작위로 분할하여 훈련(train) 및 테스트(test) 세트 생성
train_dataset, test_dataset = random_split(img_dataset, [train_size, test_size])

# 훈련(train) 및 테스트(test) 데이터 로더 생성
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [78]:
img_dataset[0]

c:\Users\lma00\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


(tensor([[[0.5569, 0.5569, 0.5569,  ..., 0.5569, 0.5569, 0.5569],
          [0.5569, 0.5569, 0.5569,  ..., 0.5569, 0.5569, 0.5569],
          [0.5569, 0.5569, 0.5569,  ..., 0.5569, 0.5569, 0.5569],
          ...,
          [0.5569, 0.5569, 0.5569,  ..., 0.5569, 0.5569, 0.5569],
          [0.5569, 0.5569, 0.5569,  ..., 0.5569, 0.5569, 0.5569],
          [0.5569, 0.5569, 0.5569,  ..., 0.5569, 0.5569, 0.5569]],
 
         [[0.5949, 0.5949, 0.5949,  ..., 0.5949, 0.5949, 0.5949],
          [0.5949, 0.5949, 0.5949,  ..., 0.5949, 0.5949, 0.5949],
          [0.5949, 0.5949, 0.5949,  ..., 0.5949, 0.5949, 0.5949],
          ...,
          [0.5949, 0.5949, 0.5949,  ..., 0.5949, 0.5949, 0.5949],
          [0.5949, 0.5949, 0.5949,  ..., 0.5949, 0.5949, 0.5949],
          [0.5949, 0.5949, 0.5949,  ..., 0.5949, 0.5949, 0.5949]],
 
         [[0.6078, 0.6078, 0.6078,  ..., 0.6078, 0.6078, 0.6078],
          [0.6078, 0.6078, 0.6078,  ..., 0.6078, 0.6078, 0.6078],
          [0.6078, 0.6078, 0.6078,  ...,

In [79]:
def get_all_filenames(directory):
    """
    Returns a set of all filenames in the given directory.
    """
    filenames = {entry.name for entry in os.scandir(directory) if entry.is_file()}
    return filenames

images = get_all_filenames('./dataset/images/')

In [80]:
train_dataset[0]

(tensor([[[0.5569, 0.5569, 0.5569,  ..., 0.5569, 0.5569, 0.5569],
          [0.5569, 0.5569, 0.5569,  ..., 0.5569, 0.5569, 0.5569],
          [0.5569, 0.5569, 0.5569,  ..., 0.5569, 0.5569, 0.5569],
          ...,
          [0.5569, 0.5569, 0.5569,  ..., 0.5569, 0.5569, 0.5569],
          [0.5569, 0.5569, 0.5569,  ..., 0.5569, 0.5569, 0.5569],
          [0.5569, 0.5569, 0.5569,  ..., 0.5569, 0.5569, 0.5569]],
 
         [[0.5949, 0.5949, 0.5949,  ..., 0.5949, 0.5949, 0.5949],
          [0.5949, 0.5949, 0.5949,  ..., 0.5949, 0.5949, 0.5949],
          [0.5949, 0.5949, 0.5949,  ..., 0.5949, 0.5949, 0.5949],
          ...,
          [0.5949, 0.5949, 0.5949,  ..., 0.5949, 0.5949, 0.5949],
          [0.5949, 0.5949, 0.5949,  ..., 0.5949, 0.5949, 0.5949],
          [0.5949, 0.5949, 0.5949,  ..., 0.5949, 0.5949, 0.5949]],
 
         [[0.6078, 0.6078, 0.6078,  ..., 0.6078, 0.6078, 0.6078],
          [0.6078, 0.6078, 0.6078,  ..., 0.6078, 0.6078, 0.6078],
          [0.6078, 0.6078, 0.6078,  ...,

In [81]:
# 모델, 손실 함수, 최적화 알고리즘 설정
model = myResNet().to(device)
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.5)
optimizer = optim.Adam(model.parameters(), lr=0.03)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [82]:
# 손실과 정확도를 저장하기 위한 리스트 초기화
train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []

In [83]:
# 훈련 함수
def train(model, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for data, target in train_loader:
        if torch.cuda.is_available():
            data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    # 예측값 계산(가장 높은 값을 가진 인덱스)
    _, predicted = torch.max(output.data, 1)
    total += target.size(0)
    # 올바르게 예측된 샘플의 수 업데이트
    correct += (predicted == target).sum().item()

    # 평균 훈련 손실 계산
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    print(f'Epoch {epoch}, Training loss: {train_loss:.4f}')
    accuracy = 100. * correct / total
    train_accuracies.append(accuracy)

In [84]:
# 테스트 함수
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            if torch.cuda.is_available():
                data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader)
    accuracy = 100. * correct / len(test_loader.dataset)
    test_losses.append(test_loss)
    test_accuracies.append(accuracy)
    print(f'Test loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%')

In [85]:
# 훈련 및 테스트 실행
for epoch in range(1, 4):
    train(model, train_data_loader, optimizer, epoch)
    test(model, test_data_loader)

Epoch 1, Training loss: 0.0088
Test loss: 0.0000, Accuracy: 100.00%
Epoch 2, Training loss: 0.0000
Test loss: 0.0000, Accuracy: 100.00%
Epoch 3, Training loss: 0.0000
Test loss: 0.0000, Accuracy: 100.00%


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# 손실 및 정확도 시각화
fig, axs = plt.subplots(2, 1, figsize=(10, 10))

axs[0].plot(train_losses, label='Training Loss')
axs[0].plot(test_losses, label='Test Loss')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')
axs[0].legend()
axs[0].set_title('Training and Test Loss')

axs[1].plot(train_accuracies, label='Training Accuracy', color='blue')
axs[1].plot(test_accuracies, label='Test Accuracy', color='orange')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Accuracy (%)')
axs[1].legend()
axs[1].set_title('Training and Test Accuracy')

plt.tight_layout()
plt.show()